# <b>Task01 - Traffic Application Classification</b>

___
## Table of Content

* Load raw features
* Train the multi-modal autoencoder
* Train the raw features models
* Validate the models
* k-nearest-neighbors class probability

In [1]:
# type: ignore

from mltoolbox.classification import DeepClassifier, KnnClassifier
from mltoolbox.representation import MultimodalAE
from sklearn.metrics import classification_report
from mltoolbox.metrics import k_class_proba_report, silhouette_report
from src.utils import get_datasets
from keras import layers
import pandas as pd
import numpy as np
import joblib

2022-12-08 12:36:10.619204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 12:36:10.813038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda3/envs/bigdatalab/lib/libfabric:/usr/local/cuda/lib64
2022-12-08 12:36:10.813071: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 12:36:10.857165: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already

In [3]:
task = 'task01' # Define the task. This variable is used mainly for savings
K = 0 # Specify the considered fold (Stratified k fold validation)

## Load raw features

This code is loading and combining multiple features from CSV files to create a new dataset. The features are loaded using the pandas library and are stored in the payload, statistics, sequences, and ipaddress dataframes. The payload dataframe contains port word2vec embeddings, the statistics dataframe contains statistical features, the sequences dataframe contains sequence features, and the ipaddress dataframe contains IP address word2vec embeddings.

The code then uses the merge method from pandas to combine the dataframes by merging them on the 'index' column, resulting in a new concat dataframe that contains all the features as a raw concatenation. Finally, the code loads stratified k-folds using the joblib library and stores them in the kfolds variable. These k-folds can be used to split the data into train and test sets for cross-validation.


In [9]:
# Load ports word2vec embeddings - entity
payload=pd.read_csv(f'../data/{task}/features/payload.csv', index_col=[0])
#payload=payload.replace({-1:0})
# Load statistics features - quantity
statistics=pd.read_csv(f'../data/{task}/features/statistics.csv', index_col=[0])
# Load statistics sequences - quantity
sequences=pd.read_csv(f'../data/{task}/features/sequences.csv', index_col=[0])
# Load ip address word2vec embeddings - entity
ipaddress=pd.read_csv(f'../data/{task}/features/ipaddress.csv', index_col=[0])
# Merge the features as raw concatenation
concat = payload.reset_index().drop(columns=['label'])\
     .merge(statistics.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(sequences.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(ipaddress.reset_index(), on='index', how='inner')\
     .set_index('index')
# Load stratified k folds
kfolds = joblib.load(f'../data/{task}/skfolds/folds.save')

## Train the multi-modal autoencoder

In this section we define a multimodal autoencoder architecture using the Keras API. The autoencoder is trained on four different input modalities: payload, statistics, sequences, and IP address embeddings. The code defines the encoder and decoder branches for each modality, and then combines them into a single autoencoder architecture. The autoencoder is trained to reconstruct the input data, which is used to learn a compact representation of the input data that can be useful for downstream tasks such as classification.

Firstly, we split the datasets into training and validation sets using the provided k-fold splits, and sets up the input and output sizes for the model based on the shape of the data. This allows the model to be trained and tested on the data.

In [5]:
feature, fname = concat, 'mae'
# Get the features size. Last column is the label one
P,St,Se,I = payload.shape[1]-1, statistics.shape[1]-1, \
            sequences.shape[1]-1, ipaddress.shape[1]-1

# Retrieve the training and validation samples from the k-folds order
X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)

The following code defines a multi-modal classifier architecture for IP address data. The architecture is composed of four encoder branches, one for each modality of the input data: payload embeddings, statistics, sequences, and IP address embeddings. The outputs of these four encoder branches are then concatenated and passed through a common encoder and bottleneck layer before being passed through four decoder branches. The decoder branches each output a reconstructed version of one of the input modalities. The classifier is trained to minimize the mean absolute error between the input and reconstructed modalities.

In [6]:
# Define the classifier architecture
inputs = layers.Input(X_train.shape[1],)

# Encoder branch of modality 1 - payload embeddings
hidden1 = layers.Lambda(lambda x: x[:, :P])(inputs)
hidden1 = layers.Reshape((-1, 1))(hidden1)
hidden1 = layers.Dense(64, activation='relu')(hidden1)
hidden1 = layers.Conv1D(32, 3, activation='relu')(hidden1)
hidden1 = layers.Dropout(.3)(hidden1)
hidden1 = layers.MaxPooling1D(2)(hidden1)
hidden1 = layers.Dropout(.3)(hidden1)
hidden1 = layers.Flatten()(hidden1)
hidden1 = layers.Dense(32, activation='relu')(hidden1)

# Encoder branch of modality 2 - statistics
hidden2 = layers.Lambda(lambda x: x[:, P:St])(inputs)
hidden2 = layers.Dense(32, activation='relu')(hidden2)

# Encoder branch of modality 3 - sequences
hidden3 = layers.Lambda(lambda x: x[:, P+St:P+St+Se])(inputs)
hidden3 = layers.Reshape((32, 4))(hidden3)
hidden3 = layers.Conv1D(32, 3, activation='relu')(hidden3)
hidden3 = layers.Dropout(.3)(hidden3)
hidden3 = layers.MaxPooling1D(2)(hidden3)
hidden3 = layers.Dropout(.3)(hidden3)
hidden3 = layers.Flatten()(hidden3)
hidden3 = layers.Dense(32, activation='relu')(hidden3)

# Encoder branch of modality 4 - ip address embeddings
hidden4 = layers.Lambda(lambda x: x[:, P+St+Se:P+St+Se+I])(inputs)
hidden4 = layers.Dense(32, activation='relu')(hidden4)

# Concatenate
hidden = layers.Concatenate()([hidden1, hidden2, hidden3, hidden4])
# Common encoder
hidden = layers.Dense(512, activation='relu')(hidden)
hidden = layers.Dense(256, activation='relu')(hidden)
# Bottleneck
hidden = layers.Dense(64, activation='relu', name='Coded')(hidden)
# Common decoder
hidden = layers.Dense(256, activation='relu')(hidden)
hidden = layers.Dense(512, activation='relu')(hidden)
hidden = layers.Dense(32*4, activation='relu')(hidden)

hidden1 = layers.Dense(32, activation='relu')(hidden)
hidden1 = layers.Dense(480, activation='relu')(hidden1)
hidden1 = layers.Reshape((15, 32))(hidden1)
hidden1 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden1)
hidden1 = layers.UpSampling1D(2)(hidden1)
hidden1 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden1)
hidden1 = layers.UpSampling1D(2)(hidden1)
hidden1 = layers.Flatten()(hidden1)
output1 = layers.Dense(P, activation='linear', name='payload')(hidden1)

hidden2 = layers.Dense(32, activation='relu')(hidden)
output2 = layers.Dense(St, activation='linear', name='statistics')(hidden2)

hidden3 = layers.Dense(32, activation='relu')(hidden)
hidden3 = layers.Dense(480, activation='relu')(hidden3)
hidden3 = layers.Reshape((15, 32))(hidden3)
hidden3 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden3)
hidden3 = layers.UpSampling1D(2)(hidden3)
hidden3 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden3)
hidden3 = layers.UpSampling1D(2)(hidden3)
hidden3 = layers.Flatten()(hidden3)
output3 = layers.Dense(Se, activation='linear', name='sequences')(hidden3)

hidden4 = layers.Dense(32, activation='relu')(hidden)
output4 = layers.Dense(I, activation='linear', name='ipaddress')(hidden4)

outputs = [output1, output2, output3, output4]

# Mean Squared Errors
loss = {'payload':'mse', 'statistics':'mse', 'sequences':'mse', 'ipaddress':'mse'} 
weights = {'payload':1/P, 'statistics':1/St, 'sequences':1/Se, 'ipaddress':1/I} # Balance losses

2022-12-07 23:26:01.284254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda3/envs/bigdatalab/lib/libfabric:/usr/local/cuda/lib64
2022-12-07 23:26:01.284321: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-07 23:26:01.284348: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vm5-bigdata): /proc/driver/nvidia/version does not exist
2022-12-07 23:26:01.284670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


This code initializes a multi-modal autoencoder (MAE) classifier and fits it to the given training data. The MAE is trained for three epochs and the resulting embeddings are stacked and saved.

In [7]:
# Initialize the classifier
mae = MultimodalAE(model_path=f'../data/{task}/mae/{fname}_k{K}',
                   io=(inputs, outputs), losses=loss, weights=weights)
# Fit the multi-modal autoencoder
mae.fit(training_data=(X_train, X_train), y_sizes=[P, St, Se, I], batch_size=256, 
        scale_data=True, epochs=3, validation_data=(X_val, X_val), save=True, 
        verbose=1)
embeddings = np.vstack([mae.transform(X_train), mae.transform(X_val)])

# Save the embeddings
embeddings = pd.DataFrame(embeddings, index=np.hstack(kfolds[K][:2]))
embeddings[['label']] = np.hstack(kfolds[K][2:])
embeddings.to_csv(f'../data/{task}/embeddings/mae_embeddings_k{K}.csv')

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
135/135 [==============================] - ETA: 0s - loss: 0.0429 - payload_loss: 0.6964 - statistics_loss: 0.5136 - sequences_loss: 0.8100 - ipaddress_loss: 0.4893WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f6204743a70> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
135/135 [==============================] - 9s 45ms/step - loss: 0.0429 - payload_loss: 0.6964 - statistics_loss: 0.5136 - sequences_loss: 0.8100 - ipaddress_loss: 0.4893 - val_loss: 0.0298 - val_payload_loss: 0.5752 - val_statistics_loss: 0.2849 - val_sequences_loss: 0.7185 - val_ipaddress_loss: 0.1458
Epoch 2/3
135/135 [==============================] - 5s 39ms/step - loss: 0.0261 - payload_loss: 0.5596 - statistics_loss: 0.1709 - sequences_loss: 0.6341 - ipaddress_loss: 0.0816 - val_loss: 0.0252 - val_payload_loss: 0.5401 - val_statistics_loss: 0.1886 - val_sequences_loss: 0.6334 - val_ipaddress_loss: 0.0499
Epoch 3/3
135/135 [==============================] - 5s 39ms/step - los

## Train the raw-features-models



The following code trains a deep classifier using different raw features datasets. The classifier is built using fully connected layers, and the supervised classification tasks are evaluated for each features set independently. The classifier architecture and number of epochs are defined and the data is standardized before training. The trained classifiers are saved in the specified directory.

In [10]:
embeddings=pd.read_csv(f'../data/{task}/embeddings/mae_embeddings_k{K}.csv', 
                       index_col=[0])
n_classes = ipaddress.value_counts('label').shape[0]

In [11]:
features = [payload, statistics, sequences, ipaddress, concat, embeddings]
feature_names = ['payload', 'statistics', 'sequences', 'ipaddress', 'rawcat', 'mae']

for feature, fname in zip(features, feature_names):
    print(f'\nTraining {fname} deep classifier')
    
    # Retrieve the training and validation samples from the k-folds order
    X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
    # Define the classifier architecture
    inputs = layers.Input(X_train.shape[1],)
    hidden = layers.Dense(512, activation='relu')(inputs)
    hidden = layers.Dropout(.3)(hidden)
    hidden = layers.Dense(256, activation='relu')(hidden)
    hidden = layers.Dropout(.3)(hidden)
    outputs = layers.Dense(n_classes, activation='softmax')(hidden)
    # Initialize the classifier
    mpath = f'../data/{task}/classifiers/{fname}_k{K}'
    classifier = DeepClassifier(io=(inputs, outputs), model_path=mpath)
    # Train the classifier for 3 epochs. Standardize data before training
    classifier.fit(training_data=(X_train, y_train), 
                   validation_data=(X_val, y_val), 
                   scale_data=True, batch_size=256, epochs=3, save=True)


Training payload deep classifier
Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
135/135 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 0.2499WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f61f409b170> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: ../data/task01/classifiers/mae_k0_classifier/assets


INFO:tensorflow:Assets written to: ../data/task01/classifiers/mae_k0_classifier/assets


135/135 [==============================] - 3s 23ms/step - loss: 3.2371e-04 - accuracy: 0.8468 - val_loss: 0.5224 - val_accuracy: 0.8489


## Validate the models

This code runs a deep classifier on several different datasets, each containing different raw features. The classification performance is evaluated on the validation set and the results are saved in a CSV file. The macro average F1 score is also printed for each dataset.

In [12]:
features = [payload, statistics, sequences, ipaddress, concat, embeddings]
feature_names = ['payload', 'statistics', 'sequences', 'ipaddress', 'rawcat', 'mae']

for feature, fname in zip(features, feature_names):
    # Retrieve the training and validation samples from the k-folds order
    X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
    mpath = f'../data/{task}/classifiers/{fname}_k{K}'
    classifier = DeepClassifier(_load_model=True, model_path=mpath)
    y_pred = classifier.predict(X_val, scale_data=True)
    report = classification_report(y_val, y_pred, labels=np.unique(y_val), 
                                   output_dict=True)

    f1 = report['macro avg']['f1-score']
    
    pd.DataFrame(report).T.to_csv(f'../data/interim/{fname}_deep_k{K}.csv')
    print(f'\nValidating {fname} deep classifier:\n\tMacro avg f1:{f1}')


Validating payload deep classifier:
	Macro avg f1:0.3597971811027275

Validating statistics deep classifier:
	Macro avg f1:0.48453570553137637

Validating sequences deep classifier:
	Macro avg f1:0.5779616749012114

Validating ipaddress deep classifier:
	Macro avg f1:0.7786323718492839

Validating rawcat deep classifier:
	Macro avg f1:0.8418235344063525

Validating mae deep classifier:
	Macro avg f1:0.8241910375600767


## k-nearest-neighbors class probability</b>

This code performs k-nearest-neighbors classification on a given dataset using two different feature sets: concat and embeddings. For each of these feature sets, it trains and evaluates a k-nearest-neighbors classifier for various values of k ranging from 1 to 20. The performance of the classifier is measured using the k-nearest-neighbors class probability (kpc). The results are saved to a CSV file and printed to the console.

In [ ]:
features = [concat, embeddings]
feature_names = ['rawcat', 'mae']
for feature, fname in zip(features, feature_names):
    for k in range(1, 20):
        # Retrieve the training and validation samples from the k-folds order
        X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
        X, y = np.vstack([X_train, X_val]), np.hstack([y_train, y_val])
        
        knn = KnnClassifier(n_neighbors=k, metric='cosine')
        knn.fit(X, y, scale_data=True)

        to_keep = np.where(y!='unknown')[0].reshape(-1, 1)
        pcs = knn.predict_proba(to_keep)
        
        y_true = y[np.ravel(to_keep)]
        report = k_class_proba_report(y_true, pcs, output_dict=True)
        
        kpc = report['macro avg']['kpc']

        pd.DataFrame(report).T.to_csv(f'../data/interim/{fname}_{k}pc_k{K}.csv')
        print(f'\nMacro avg {k}Pc - {fname}: {kpc}')

## Shallow learners

This code trains and evaluates a Random Forest classifier and a kNN classifier on a given dataset. The performance of the classifier is measured using the macro average f1-score. The results are saved to a CSV file and printed to the console. The purpose of this code is to check whether the features generated by a multimodal autoencoder can be exploited by a shallow learning classifier, in addition to the deep learning classifiers that have already been tried.

In [4]:
embeddings=pd.read_csv(f'../data/{task}/embeddings/mae_embeddings_k{K}.csv', 
                       index_col=[0])
n_classes = embeddings.value_counts('label').shape[0]

X_train, X_val, y_train, y_val = get_datasets(kfolds, K, embeddings)

We firstly try Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

report = classification_report(y_val, y_pred, labels=np.unique(y_val), 
                                   output_dict=True)

f1 = report['macro avg']['f1-score']

pd.DataFrame(report).T.to_csv(f'../data/interim/mae_rf_k{K}.csv')
print(f'\nValidating MAE Random Forest classifier:\n\tMacro avg f1:{f1}')


Validating MAE Random Forest classifier:
	Macro avg f1:0.8482466243920457


Then, a kNN classifier

In [6]:
from mltoolbox.classification import KnnClassifier

clf = KnnClassifier(n_neighbors=7, metric='cosine')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

report = classification_report(y_val, y_pred, labels=np.unique(y_val), 
                                   output_dict=True)

f1 = report['macro avg']['f1-score']

pd.DataFrame(report).T.to_csv(f'../data/interim/mae_7nn_k{K}.csv')
print(f'\nValidating MAE Random Forest classifier:\n\tMacro avg f1:{f1}')

/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Validating MAE Random Forest classifier:
	Macro avg f1:0.8254661435593664


## Unsupervised clustering

In our previous experiments, we tested deep and shallow learning models for a supervised learning task. We are now interested in evaluating whether it is possible to use the generated embeddings for an unsupervised learning task. To do this, we will use clustering algorithms to see if the generated embeddings can be used to group similar data points together. We will evaluate the performance of the clustering using various metrics to determine its effectiveness.

In [16]:
from mltoolbox.clustering import kMeans
from mltoolbox.metrics import silhouette_report
from sklearn.metrics import adjusted_rand_score


This snippet is for evaluating clustering performance using two metrics: silhouette coefficient and adjusted rand index. It clusters data using the kMeans method with different numbers of clusters (kks) and outputs the evaluation metrics for each. The get_datasets function is used to obtain the input data, which is then split into training and validation sets. The silhouette_report function is used to calculate the silhouette coefficient, and the adjusted_rand_score function is used to calculate the adjusted rand index. The final results are printed to the console.

In [21]:
kks = range(8,33)

X_train, X_val, y_train, y_val = get_datasets(kfolds, K, embeddings)
X = np.vstack([X_train, X_val])
y = np.ravel(np.hstack([y_train, y_val]))

X, y = X[:100], y[:100]

shs_mae, ari_mae = [], []
for kk in kks:
    kmeans = kMeans(n_clusters=kk)
    kmeans.fit(X, scale_data=False)
    y_pred = kmeans.predict(X, scale_data=False)
    report = silhouette_report(X, y_pred, output_dict=True)
    sh = report['macro avg']['sh']
    shs_mae.append(sh)
    ari = adjusted_rand_score(y, y_pred)
    ari_mae.append(ari)
    print(f'\nMAE clustering k{kk}:\n\tMacro avg Sh:{sh}, ARI:{ari}')


MAE clustering k8:
	Macro avg Sh:0.43, ARI:0.0

MAE clustering k9:
	Macro avg Sh:0.43, ARI:0.0

MAE clustering k10:
	Macro avg Sh:0.31, ARI:0.0

MAE clustering k11:
	Macro avg Sh:0.31, ARI:0.0

MAE clustering k12:
	Macro avg Sh:0.29, ARI:0.0

MAE clustering k13:
	Macro avg Sh:0.24, ARI:0.0

MAE clustering k14:
	Macro avg Sh:0.23, ARI:0.0

MAE clustering k15:
	Macro avg Sh:0.26, ARI:0.0

MAE clustering k16:
	Macro avg Sh:0.29, ARI:0.0

MAE clustering k17:
	Macro avg Sh:0.27, ARI:0.0

MAE clustering k18:
	Macro avg Sh:0.22, ARI:0.0

MAE clustering k19:
	Macro avg Sh:0.25, ARI:0.0

MAE clustering k20:
	Macro avg Sh:0.26, ARI:0.0

MAE clustering k21:
	Macro avg Sh:0.23, ARI:0.0

MAE clustering k22:
	Macro avg Sh:0.24, ARI:0.0

MAE clustering k23:
	Macro avg Sh:0.24, ARI:0.0

MAE clustering k24:
	Macro avg Sh:0.21, ARI:0.0

MAE clustering k25:
	Macro avg Sh:0.25, ARI:0.0

MAE clustering k26:
	Macro avg Sh:0.25, ARI:0.0

MAE clustering k27:
	Macro avg Sh:0.21, ARI:0.0

MAE clustering k28:
	

In [23]:
X_train, X_val, y_train, y_val = get_datasets(kfolds, K, concat)
X = np.vstack([X_train, X_val])
y = np.ravel(np.hstack([y_train, y_val]))

X, y = X[:100], y[:100]

shs_rawcat, ari_rawcat = [], []
for kk in kks:
    kmeans = kMeans(n_clusters=kk)
    kmeans.fit(X, scale_data=False)
    y_pred = kmeans.predict(X, scale_data=False)
    report = silhouette_report(X, y_pred, output_dict=True)
    sh = report['macro avg']['sh']
    shs_rawcat.append(sh)
    ari = adjusted_rand_score(y, y_pred)
    ari_rawcat.append(ari)
    print(f'\nRawCat clustering k{kk}:\n\tMacro avg Sh:{sh}, ARI:{ari}')


RawCat clustering k8:
	Macro avg Sh:0.44, ARI:0.0

RawCat clustering k9:
	Macro avg Sh:0.41, ARI:0.0

RawCat clustering k10:
	Macro avg Sh:0.4, ARI:0.0

RawCat clustering k11:
	Macro avg Sh:0.41, ARI:0.0

RawCat clustering k12:
	Macro avg Sh:0.4, ARI:0.0

RawCat clustering k13:
	Macro avg Sh:0.4, ARI:0.0

RawCat clustering k14:
	Macro avg Sh:0.42, ARI:0.0

RawCat clustering k15:
	Macro avg Sh:0.41, ARI:0.0

RawCat clustering k16:
	Macro avg Sh:0.41, ARI:0.0

RawCat clustering k17:
	Macro avg Sh:0.35, ARI:0.0

RawCat clustering k18:
	Macro avg Sh:0.42, ARI:0.0

RawCat clustering k19:
	Macro avg Sh:0.4, ARI:0.0

RawCat clustering k20:
	Macro avg Sh:0.38, ARI:0.0

RawCat clustering k21:
	Macro avg Sh:0.4, ARI:0.0

RawCat clustering k22:
	Macro avg Sh:0.36, ARI:0.0

RawCat clustering k23:
	Macro avg Sh:0.33, ARI:0.0

RawCat clustering k24:
	Macro avg Sh:0.32, ARI:0.0

RawCat clustering k25:
	Macro avg Sh:0.35, ARI:0.0

RawCat clustering k26:
	Macro avg Sh:0.39, ARI:0.0

RawCat clustering 

In [27]:
sh_df = pd.DataFrame(zip(shs_mae, shs_rawcat), 
                     index=kks, columns=['sh_mae', 'sh_cat'])
ar_df = pd.DataFrame(zip(ari_mae, ari_rawcat), 
                     index=kks, columns=['ari_mae', 'ari_cat'])

sh_df = sh_df.reset_index().merge(ar_df.reset_index(), on='index')\
             .set_index('index')
sh_df.to_csv(f'../data/interim/clustering_k{K}.csv')

sh_df.head()

,sh_mae,sh_cat,ari_mae,ari_cat
index,,,,
8,0.43,0.44,0.0,0.0
9,0.43,0.41,0.0,0.0
10,0.31,0.40,0.0,0.0
11,0.31,0.41,0.0,0.0
12,0.29,0.40,0.0,0.0
